## imports

In [ ]:
!pip install torch_geometric

In [ ]:
!pip install networkx

In [ ]:
from torch_geometric.datasets import MNISTSuperpixels, CitationFull
import numpy as np

In [ ]:
import os
import torch
from torch_geometric.datasets import MNISTSuperpixels
from torch_geometric.utils import to_scipy_sparse_matrix
import scipy.sparse as sp
from scipy.sparse.csgraph import laplacian
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MNISTSuperpixels, CitationFull, WebKB, Actor, WikipediaNetwork


In [ ]:
if not os.path.exists('dataset/'):
    os.makedirs('dataset/')

# utils

In [ ]:
# Function to compute the Laplacian matrix of a graph from PyTorch Geometric
def compute_laplacian_pyg(data, normalized=True):
    """
    Compute the Laplacian matrix for a PyTorch Geometric Data object.
    """
    edge_index = data.edge_index
    num_nodes = data.num_nodes

    # Convert edge_index to a SciPy sparse matrix
    adj_matrix = to_scipy_sparse_matrix(edge_index, num_nodes=num_nodes)

    # Compute the Laplacian
    L = laplacian(adj_matrix, normed=normalized)

    return L

def laplacian_spectrum(L):
    eigenvals, eigenvecs = np.linalg.eigh(L)
    idx = eigenvals.argsort()
    eigenvals = eigenvals[idx]
    eigenvecs = eigenvecs[:,idx]
    def transform(x, i=0, k=None):
        if k is None:
            k = len(x)
        return eigenvecs[:,i:i+k].T.dot(x)
    return transform, eigenvals

def laplacian_spectrum_full(L):
    eigenvals, eigenvecs = np.linalg.eigh(L)
    idx = eigenvals.argsort()
    eigenvals = eigenvals[idx]
    eigenvecs = eigenvecs[:,idx]
    def transform(x, i=0, k=None):
        if k is None:
            k = len(x)
        return eigenvecs[:,i:i+k].T.dot(x)
    def inv_transform(xh, i=0, k=None):
        if k is None:
            k = len(xh)
        return eigenvecs[:,i:i+k].dot(xh)
    return transform, inv_transform, eigenvals

def compute_frequency(dataset_instance, dataset_name):
    """used to plot the frequency distribution of the node classification datasets"""
    L = compute_laplacian_pyg(dataset_instance, normalized=True)
    L_array = L.toarray()
    print("computing transorm and eigenvalues...")
    transform, eigenvals = laplacian_spectrum(L_array)

    x = dataset_instance.x.numpy()

    x_transformed = transform(x)

    dataset_x_sum = np.sum(np.abs(x_transformed), axis=1)


    plt.figure(figsize=(7, 5))
    plt.stem(eigenvals, dataset_x_sum, use_line_collection=True)
    plt.title('Summed Distribution of Frequency Components Across All Features in '+ dataset_name)
    plt.xlim([-0.05, 2])
    plt.xlabel('Eigenvalue (Frequency)')
    plt.ylabel('Summed Magnitude')
    plt.grid(True)
    plt.show()
    filename = dataset_name.replace(" ", "_") + '_Frequency_Distribution.png'
    # plt.savefig(filename,dpi=600)  # Save the plot as a PNG file
    print(f'Plot saved as {filename}')
    plt.close()


def plot_graphs_from_dataset(dataset, num_examples=3):
    """use to plot some examples from graph classification datasets"""
    plt.figure(figsize=(15, 5))

    for i in range(min(num_examples, len(dataset))):
        graph = dataset[i]
        nx_graph = to_networkx(graph, to_undirected=True)

        plt.subplot(1, num_examples, i+1)
        nx.draw(nx_graph, with_labels=True, node_color='skyblue', edge_color='k', node_size=700, font_size=10)
        plt.title(f'Graph {i+1}')

    plt.show()


# semi-supervised node classification on Homophilic graphs

## Datasets

In [ ]:
# dataset_mnist = MNISTSuperpixels(root='dataset/MNIST/', train=True)
dataset_cora = CitationFull(root='dataset/Cora/', name = "Cora")
dataset_citeseer = CitationFull(root='dataset/CiteSeer/', name = "CiteSeer")
dataset_pubmed = CitationFull(root='dataset/PubMed/', name = "PubMed")

In [ ]:
compute_frequency(dataset_cora[0], 'Cora')

computing transorm and eigenvalues...
Plot saved as Cora_Frequency_Distribution.png


In [ ]:
compute_frequency(dataset_citeseer[0], 'Citeseer')

computing transorm and eigenvalues...
Plot saved as Citeseer_Frequency_Distribution.png


In [ ]:
compute_frequency(dataset_pubmed[0], 'Pubmed')

computing transorm and eigenvalues...
Plot saved as Pubmed_Frequency_Distribution.png


## Define GCN and ChebNet

In [ ]:

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCNModel_semi(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCNModel_semi, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

from torch_geometric.nn import ChebConv

class ChebNetModel_semi(torch.nn.Module):
    def __init__(self, num_features, num_classes, K=3):
        super(ChebNetModel_semi, self).__init__()
        self.cheb1 = ChebConv(num_features, 16, K)
        self.cheb2 = ChebConv(16, num_classes, K)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.cheb1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.cheb2(x, edge_index)
        return F.log_softmax(x, dim=1)


In [ ]:
def train(model, optimizer, data, criterion, device='cuda'):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def test(model, data, device='cuda'):
    model.eval()
    logits, accs = model(data), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        pred = logits[mask].max(1)[1]
        acc = pred.eq(data.y[mask]).sum().item() / mask.sum().item()
        accs.append(acc)
    return accs


In [ ]:
from torch_geometric.data import DataLoader

import torch

def add_train_test_masks_to_dataset(dataset):
    num_nodes = dataset.data.num_nodes
    num_train = int(0.8 * num_nodes)
    num_test = num_nodes - num_train

    # Create masks
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    # Randomly select indices for train/test
    indices = torch.randperm(num_nodes)
    train_indices, test_indices = indices[:num_train], indices[num_train:]

    train_mask[train_indices] = True
    test_mask[test_indices] = True

    # Add masks to the dataset
    dataset.data.train_mask = train_mask
    dataset.data.test_mask = test_mask

    return dataset


def data_load_sep_for_node_classification(dataset):
    """get feasures of dataset"""
    data = dataset.data

    if hasattr(data, 'train_mask') and hasattr(data, 'test_mask'):
        train_mask = data.train_mask
        test_mask = data.test_mask
    else:
        raise AttributeError("Dataset does not have 'train_mask' and 'test_mask' attributes.")

    # DataLoader setup
    loader = DataLoader(dataset, batch_size=1, shuffle=False)  # Single batch for the whole graph

    num_features = dataset.num_features
    num_classes = dataset.num_classes

    # Return the loader and masks
    return num_features, num_classes, loader, train_mask, test_mask

In [ ]:

def run_experiment_semi_supervised(model_class, optimizer_class, dataset, criterion, device="cuda", num_runs=5, epochs=200):
    all_losses = []  # To store losses from all runs
    accuracies = []
    data = dataset[0].to(device)  # Assuming a single-graph dataset

    # Masks are assumed to be part of the dataset
    train_mask = data.train_mask
    test_mask = data.test_mask

    for run in range(num_runs):
        model = model_class(dataset.num_features, dataset.num_classes).to(device)
        optimizer = optimizer_class(model.parameters(), lr=0.01)
        run_losses = []  # To store losses for the current run

        # Train the model
        for epoch in range(epochs):
            model.train()
            optimizer.zero_grad()
            out = model(data)
            loss = criterion(out[train_mask], data.y[train_mask])
            loss.backward()
            optimizer.step()

            if epoch % 10 == 0:  # Record loss every 10 epochs
                run_losses.append(loss.item())
                print(f'Run {run + 1}, Epoch {epoch}: Loss {loss.item():.4f}')

        all_losses.append(run_losses)  # Store the losses for this run

        # Test the model
        model.eval()
        _, pred = model(data).max(dim=1)
        correct = float(pred[test_mask].eq(data.y[test_mask]).sum().item())
        acc = correct / test_mask.sum().item()
        accuracies.append(acc)

        print(f'Run {run + 1}: Model Test Accuracy: {acc:.4f}')

    avg_accuracy = sum(accuracies) / num_runs
    print(f'Average Test Accuracy over {num_runs} runs: {avg_accuracy:.4f}')
    return avg_accuracy, all_losses



def data_load_sep(dataset):
    dataset = dataset.shuffle()

    train_size = int(len(dataset) * 0.8)  # 80% for training
    test_size = len(dataset) - train_size

    train_dataset, test_dataset = dataset[:train_size], dataset[train_size:]

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    num_features = train_dataset.num_features
    num_classes = train_dataset.num_classes

    return num_features, num_classes,train_loader, test_loader


In [ ]:
import torch

def add_train_val_test_masks_to_dataset(dataset, train_percent=0.7, val_percent=0.1):
    """
    Add train, validation, and test masks to the dataset for semi-supervised learning.

    Parameters:
    - dataset: The dataset object, assumed to contain a single graph.
    - train_percent: The percentage of nodes used for training.
    - val_percent: The percentage of nodes used for validation.
    """
    num_nodes = dataset.data.num_nodes
    num_train = int(train_percent * num_nodes)
    num_val = int(val_percent * num_nodes)
    num_test = num_nodes - num_train - num_val

    # Initialize masks
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)

    # Ensure labels are evenly distributed across splits
    labels = dataset.data.y.cpu().numpy()
    unique_labels = torch.unique(dataset.data.y).cpu().numpy()

    for label in unique_labels:
        label_indices = torch.where(dataset.data.y == label)[0]
        # Shuffle indices of the current label
        label_indices = label_indices[torch.randperm(len(label_indices))]

        # Calculate split sizes for the current label
        num_label_train = int(train_percent * len(label_indices))
        num_label_val = int(val_percent * len(label_indices))

        # Assign splits for the current label
        train_mask[label_indices[:num_label_train]] = True
        val_mask[label_indices[num_label_train:num_label_train + num_label_val]] = True
        test_mask[label_indices[num_label_train + num_label_val:]] = True

    # Add masks to the dataset
    dataset.data.train_mask = train_mask
    dataset.data.val_mask = val_mask
    dataset.data.test_mask = test_mask

    return dataset


In [ ]:
from torch_geometric.data import DataLoader

def data_load_sep_for_semi_supervised(dataset):
    """
    data loaders and masks for semi-supervised node classification task.
    """
    data = dataset.data

    # Check for required masks
    required_masks = ['train_mask', 'val_mask', 'test_mask']
    for mask_name in required_masks:
        if not hasattr(data, mask_name):
            raise AttributeError(f"Dataset does not have '{mask_name}'. Please add it before calling this function.")
    train_mask = data.train_mask
    val_mask = data.val_mask
    test_mask = data.test_mask

    loader = DataLoader(dataset, batch_size=1, shuffle=False)  # Process the whole graph at once

    num_features = dataset.num_features
    num_classes = dataset.num_classes

    return num_features, num_classes, loader, train_mask, val_mask, test_mask

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
criterion = torch.nn.CrossEntropyLoss()


In [ ]:
import matplotlib.pyplot as plt

def plot_losses(losses1, losses2, label1='Run 1 Loss', label2='Run 2 Loss'):
    """
    Plots the training losses from two experimental runs.

    Parameters:
    - losses1: List of losses from the first run.
    - losses2: List of losses from the second run.
    - label1: Label for the first run.
    - label2: Label for the second run.
    """
    plt.figure(figsize=(10, 6))
    plt.plot(losses1, label=label1, marker='o')
    plt.plot(losses2, label=label2, marker='x')
    plt.title('Training Losses Comparison')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()


# Cora

In [ ]:
dataset_cora2 = add_train_val_test_masks_to_dataset(dataset_cora)

## GCN

In [ ]:
avg_accuracy, loss_cora = run_experiment_semi_supervised(GCNModel_semi, torch.optim.Adam,dataset_cora2, criterion)


Run 1, Epoch 0: Loss 1.0660
Run 1, Epoch 10: Loss 0.5257
Run 1, Epoch 20: Loss 0.4254
Run 1, Epoch 30: Loss 0.3875
Run 1, Epoch 40: Loss 0.3563
Run 1, Epoch 50: Loss 0.3418
Run 1, Epoch 60: Loss 0.3251
Run 1, Epoch 70: Loss 0.3126
Run 1, Epoch 80: Loss 0.2988
Run 1, Epoch 90: Loss 0.2941
Run 1, Epoch 100: Loss 0.2819
Run 1, Epoch 110: Loss 0.2784
Run 1, Epoch 120: Loss 0.2717
Run 1, Epoch 130: Loss 0.2658
Run 1, Epoch 140: Loss 0.2632
Run 1, Epoch 150: Loss 0.2545
Run 1, Epoch 160: Loss 0.2492
Run 1, Epoch 170: Loss 0.2467
Run 1, Epoch 180: Loss 0.2397
Run 1, Epoch 190: Loss 0.2363
Run 1: Model Test Accuracy: 0.8786
Run 2, Epoch 0: Loss 1.1047
Run 2, Epoch 10: Loss 0.5239
Run 2, Epoch 20: Loss 0.4380
Run 2, Epoch 30: Loss 0.4032
Run 2, Epoch 40: Loss 0.3695
Run 2, Epoch 50: Loss 0.3522
Run 2, Epoch 60: Loss 0.3390
Run 2, Epoch 70: Loss 0.3224
Run 2, Epoch 80: Loss 0.3140
Run 2, Epoch 90: Loss 0.3076
Run 2, Epoch 100: Loss 0.2971
Run 2, Epoch 110: Loss 0.2939
Run 2, Epoch 120: Loss 0.28

## GhebNet

In [ ]:
avg_accuracy_cheb_cora, loss_cora_cheb = run_experiment_semi_supervised(ChebNetModel_semi, torch.optim.Adam,dataset_cora2, criterion)


Run 1, Epoch 0: Loss 1.5637
Run 1, Epoch 10: Loss 0.8667
Run 1, Epoch 20: Loss 0.7281
Run 1, Epoch 30: Loss 0.5681
Run 1, Epoch 40: Loss 0.4478
Run 1, Epoch 50: Loss 0.3998
Run 1, Epoch 60: Loss 0.3723
Run 1, Epoch 70: Loss 0.3483
Run 1, Epoch 80: Loss 0.3325
Run 1, Epoch 90: Loss 0.3185
Run 1, Epoch 100: Loss 0.2954
Run 1, Epoch 110: Loss 0.2953
Run 1, Epoch 120: Loss 0.2827
Run 1, Epoch 130: Loss 0.2745
Run 1, Epoch 140: Loss 0.2682
Run 1, Epoch 150: Loss 0.2612
Run 1, Epoch 160: Loss 0.2485
Run 1, Epoch 170: Loss 0.2444
Run 1, Epoch 180: Loss 0.2378
Run 1, Epoch 190: Loss 0.2361
Run 1: Model Test Accuracy: 0.8842
Run 2, Epoch 0: Loss 1.5137
Run 2, Epoch 10: Loss 0.6887
Run 2, Epoch 20: Loss 0.5555
Run 2, Epoch 30: Loss 0.4772
Run 2, Epoch 40: Loss 0.4271
Run 2, Epoch 50: Loss 0.4047
Run 2, Epoch 60: Loss 0.3870
Run 2, Epoch 70: Loss 0.3682
Run 2, Epoch 80: Loss 0.3614
Run 2, Epoch 90: Loss 0.3422
Run 2, Epoch 100: Loss 0.3316
Run 2, Epoch 110: Loss 0.3176
Run 2, Epoch 120: Loss 0.30

# Citeseer

In [ ]:
dataset_citeseer2 = add_train_val_test_masks_to_dataset(dataset_citeseer)

## GCN

In [ ]:
avg_accuracy_gcn_citeseer, loss_citeseer = run_experiment_semi_supervised(GCNModel_semi, torch.optim.Adam,dataset_citeseer2, criterion)


Run 1, Epoch 0: Loss 1.0660
Run 1, Epoch 10: Loss 0.5257
Run 1, Epoch 20: Loss 0.4254
Run 1, Epoch 30: Loss 0.3875
Run 1, Epoch 40: Loss 0.3563
Run 1, Epoch 50: Loss 0.3418
Run 1, Epoch 60: Loss 0.3251
Run 1, Epoch 70: Loss 0.3126
Run 1, Epoch 80: Loss 0.2988
Run 1, Epoch 90: Loss 0.2941
Run 1, Epoch 100: Loss 0.2819
Run 1, Epoch 110: Loss 0.2784
Run 1, Epoch 120: Loss 0.2717
Run 1, Epoch 130: Loss 0.2658
Run 1, Epoch 140: Loss 0.2632
Run 1, Epoch 150: Loss 0.2545
Run 1, Epoch 160: Loss 0.2492
Run 1, Epoch 170: Loss 0.2467
Run 1, Epoch 180: Loss 0.2397
Run 1, Epoch 190: Loss 0.2363
Run 1: Model Test Accuracy: 0.8786
Run 2, Epoch 0: Loss 1.1047
Run 2, Epoch 10: Loss 0.5239
Run 2, Epoch 20: Loss 0.4380
Run 2, Epoch 30: Loss 0.4032
Run 2, Epoch 40: Loss 0.3695
Run 2, Epoch 50: Loss 0.3522
Run 2, Epoch 60: Loss 0.3390
Run 2, Epoch 70: Loss 0.3224
Run 2, Epoch 80: Loss 0.3140
Run 2, Epoch 90: Loss 0.3076
Run 2, Epoch 100: Loss 0.2971
Run 2, Epoch 110: Loss 0.2939
Run 2, Epoch 120: Loss 0.28

## GhebNet

In [ ]:
avg_accuracy_cheb_citeer, loss_cheb_citeseer = run_experiment_semi_supervised(ChebNetModel_semi, torch.optim.Adam,dataset_citeseer2, criterion)


Run 1, Epoch 0: Loss 1.5637
Run 1, Epoch 10: Loss 0.8667
Run 1, Epoch 20: Loss 0.7281
Run 1, Epoch 30: Loss 0.5681
Run 1, Epoch 40: Loss 0.4478
Run 1, Epoch 50: Loss 0.3998
Run 1, Epoch 60: Loss 0.3723
Run 1, Epoch 70: Loss 0.3483
Run 1, Epoch 80: Loss 0.3325
Run 1, Epoch 90: Loss 0.3185
Run 1, Epoch 100: Loss 0.2954
Run 1, Epoch 110: Loss 0.2953
Run 1, Epoch 120: Loss 0.2827
Run 1, Epoch 130: Loss 0.2745
Run 1, Epoch 140: Loss 0.2682
Run 1, Epoch 150: Loss 0.2612
Run 1, Epoch 160: Loss 0.2485
Run 1, Epoch 170: Loss 0.2444
Run 1, Epoch 180: Loss 0.2378
Run 1, Epoch 190: Loss 0.2361
Run 1: Model Test Accuracy: 0.8842
Run 2, Epoch 0: Loss 1.5137
Run 2, Epoch 10: Loss 0.6887
Run 2, Epoch 20: Loss 0.5555
Run 2, Epoch 30: Loss 0.4772
Run 2, Epoch 40: Loss 0.4271
Run 2, Epoch 50: Loss 0.4047
Run 2, Epoch 60: Loss 0.3870
Run 2, Epoch 70: Loss 0.3682
Run 2, Epoch 80: Loss 0.3614
Run 2, Epoch 90: Loss 0.3422
Run 2, Epoch 100: Loss 0.3316
Run 2, Epoch 110: Loss 0.3176
Run 2, Epoch 120: Loss 0.30

# Pumbed

In [ ]:
dataset_pubmed2 = add_train_val_test_masks_to_dataset(dataset_pubmed)

## GCN

In [ ]:
avg_accuracy_gcn_pumbed, loss_gcn_pumbed = run_experiment_semi_supervised(GCNModel_semi, torch.optim.Adam,dataset_pubmed2, criterion)


Run 1, Epoch 0: Loss 1.0660
Run 1, Epoch 10: Loss 0.5257
Run 1, Epoch 20: Loss 0.4254
Run 1, Epoch 30: Loss 0.3875
Run 1, Epoch 40: Loss 0.3563
Run 1, Epoch 50: Loss 0.3418
Run 1, Epoch 60: Loss 0.3251
Run 1, Epoch 70: Loss 0.3126
Run 1, Epoch 80: Loss 0.2988
Run 1, Epoch 90: Loss 0.2941
Run 1, Epoch 100: Loss 0.2819
Run 1, Epoch 110: Loss 0.2784
Run 1, Epoch 120: Loss 0.2717
Run 1, Epoch 130: Loss 0.2658
Run 1, Epoch 140: Loss 0.2632
Run 1, Epoch 150: Loss 0.2545
Run 1, Epoch 160: Loss 0.2492
Run 1, Epoch 170: Loss 0.2467
Run 1, Epoch 180: Loss 0.2397
Run 1, Epoch 190: Loss 0.2363
Run 1: Model Test Accuracy: 0.8786
Run 2, Epoch 0: Loss 1.1047
Run 2, Epoch 10: Loss 0.5239
Run 2, Epoch 20: Loss 0.4380
Run 2, Epoch 30: Loss 0.4032
Run 2, Epoch 40: Loss 0.3695
Run 2, Epoch 50: Loss 0.3522
Run 2, Epoch 60: Loss 0.3390
Run 2, Epoch 70: Loss 0.3224
Run 2, Epoch 80: Loss 0.3140
Run 2, Epoch 90: Loss 0.3076
Run 2, Epoch 100: Loss 0.2971
Run 2, Epoch 110: Loss 0.2939
Run 2, Epoch 120: Loss 0.28

## GhebNet

In [ ]:
avg_accuracy_cheb_pumbed, loss_cheb_pumbed = run_experiment_semi_supervised(ChebNetModel_semi, torch.optim.Adam,dataset_pubmed2, criterion)


Run 1, Epoch 0: Loss 1.5637
Run 1, Epoch 10: Loss 0.8667
Run 1, Epoch 20: Loss 0.7281
Run 1, Epoch 30: Loss 0.5681
Run 1, Epoch 40: Loss 0.4478
Run 1, Epoch 50: Loss 0.3998
Run 1, Epoch 60: Loss 0.3723
Run 1, Epoch 70: Loss 0.3483
Run 1, Epoch 80: Loss 0.3325
Run 1, Epoch 90: Loss 0.3185
Run 1, Epoch 100: Loss 0.2954
Run 1, Epoch 110: Loss 0.2953
Run 1, Epoch 120: Loss 0.2827
Run 1, Epoch 130: Loss 0.2745
Run 1, Epoch 140: Loss 0.2682
Run 1, Epoch 150: Loss 0.2612
Run 1, Epoch 160: Loss 0.2485
Run 1, Epoch 170: Loss 0.2444
Run 1, Epoch 180: Loss 0.2378
Run 1, Epoch 190: Loss 0.2361
Run 1: Model Test Accuracy: 0.8842
Run 2, Epoch 0: Loss 1.5137
Run 2, Epoch 10: Loss 0.6887
Run 2, Epoch 20: Loss 0.5555
Run 2, Epoch 30: Loss 0.4772
Run 2, Epoch 40: Loss 0.4271
Run 2, Epoch 50: Loss 0.4047
Run 2, Epoch 60: Loss 0.3870
Run 2, Epoch 70: Loss 0.3682
Run 2, Epoch 80: Loss 0.3614
Run 2, Epoch 90: Loss 0.3422
Run 2, Epoch 100: Loss 0.3316
Run 2, Epoch 110: Loss 0.3176
Run 2, Epoch 120: Loss 0.30